### Capture Matrix Construction using IEC61400-13

#### 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import datetime, copy
%matplotlib inline

#### 2. Nice representation of DataFrame in Markdown

In [2]:
from IPython.display import Markdown, display

def df2md(df):    
    fmt = ['---' for i in range(len(df.columns))]
    df_fmt = pd.DataFrame([fmt], columns=df.columns)
    df_formatted = pd.concat([df_fmt, df])
    display(Markdown(df_formatted.to_csv(sep="|", index=False)))

#### 3. Read Data

In [3]:
infilename = '../measData_Hyosung_0118-0218.xls'
data = pd.read_excel(infilename)
data = data.iloc[:, 0:8]
df2md(data.head())

month|day|hour|minute|windDirValidation|powerValidation|meanWindSpeed|meanWindSpeedTenMinutes
---|---|---|---|---|---|---|---
1|18|0|0|1|0|10.355213963333378|9.092461652000043
1|18|0|1|1|0|9.321983706666611|9.098565504333349
1|18|0|2|1|0|9.699291810000195|9.210747028000025
1|18|0|3|1|0|8.507020326666794|9.232126642666683
1|18|0|4|1|0|9.689152010000056|9.329700709


#### 4. Rename columns

In [4]:
data.rename(columns={
                     'windDirValidation':'dir', 
                     'powerValidation':'pow', 
                     'meanWindSpeed':'mean1',
                     'meanWindSpeedTenMinutes':'mean10'
                      }, inplace=True)
df2md(data.head())

month|day|hour|minute|dir|pow|mean1|mean10
---|---|---|---|---|---|---|---
1|18|0|0|1|0|10.355213963333378|9.092461652000043
1|18|0|1|1|0|9.321983706666611|9.098565504333349
1|18|0|2|1|0|9.699291810000195|9.210747028000025
1|18|0|3|1|0|8.507020326666794|9.232126642666683
1|18|0|4|1|0|9.689152010000056|9.329700709


#### 5. Check data type

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46021 entries, 0 to 46020
Data columns (total 8 columns):
month     46021 non-null int64
day       46021 non-null int64
hour      46021 non-null int64
minute    46021 non-null int64
dir       46021 non-null int64
pow       46021 non-null int64
mean1     46021 non-null float64
mean10    46021 non-null float64
dtypes: float64(2), int64(6)
memory usage: 2.8 MB


#### 6. Create Timestamp from split data

In [6]:
data['year'] = 2019
data['dateAndTime'] = data[['year', 'month', 'day', 'hour', 'minute']].apply(lambda s: str(datetime.datetime(*s)), axis=1)
df2md(data.head())

month|day|hour|minute|dir|pow|mean1|mean10|year|dateAndTime
---|---|---|---|---|---|---|---|---|---
1|18|0|0|1|0|10.355213963333378|9.092461652000043|2019|2019-01-18 00:00:00
1|18|0|1|1|0|9.321983706666611|9.098565504333349|2019|2019-01-18 00:01:00
1|18|0|2|1|0|9.699291810000195|9.210747028000025|2019|2019-01-18 00:02:00
1|18|0|3|1|0|8.507020326666794|9.232126642666683|2019|2019-01-18 00:03:00
1|18|0|4|1|0|9.689152010000056|9.329700709|2019|2019-01-18 00:04:00


#### 7. Calculate indexMinutes

In [7]:
data['indexMinutes']= data['minute'] + 60*data['hour'] + 1440*(data['day']-18) + 44640*(data['month']-1)
df2md(data.head())

month|day|hour|minute|dir|pow|mean1|mean10|year|dateAndTime|indexMinutes
---|---|---|---|---|---|---|---|---|---|---
1|18|0|0|1|0|10.355213963333378|9.092461652000043|2019|2019-01-18 00:00:00|0
1|18|0|1|1|0|9.321983706666611|9.098565504333349|2019|2019-01-18 00:01:00|1
1|18|0|2|1|0|9.699291810000195|9.210747028000025|2019|2019-01-18 00:02:00|2
1|18|0|3|1|0|8.507020326666794|9.232126642666683|2019|2019-01-18 00:03:00|3
1|18|0|4|1|0|9.689152010000056|9.329700709|2019|2019-01-18 00:04:00|4


* **Check validity of indexMinutes**

In [8]:
for i in range(10000):
    if data['indexMinutes'].iloc[i+1] - data['indexMinutes'].iloc[i] != 1:
        print(data.iloc[i-1:i+3, 4:])

      dir  pow      mean1     mean10  year          dateAndTime  indexMinutes
5408    1    0  12.105328  12.730868  2019  2019-01-21 18:08:00          5408
5409    1    0  12.642123  12.771113  2019  2019-01-21 18:09:00          5409
5410    1    0  13.257978  12.790544  2019  2019-01-21 18:15:00          5415
5411    1    0  13.186538  12.713785  2019  2019-01-21 18:16:00          5416
      dir  pow      mean1     mean10  year          dateAndTime  indexMinutes
5748    0    0  11.685295  11.969218  2019  2019-01-21 23:53:00          5753
5749    0    0  12.517311  11.904316  2019  2019-01-21 23:54:00          5754
5750    0    0  11.481346  11.841086  2019  2019-01-22 00:00:00          5760
5751    0    0  10.568442  12.006772  2019  2019-01-22 00:01:00          5761


#### 8. First Data of each date

In [9]:
first_data1 = data.groupby(['month', 'day']).first().reset_index()
first_data2 = data.groupby(['month', 'day']).apply(lambda x: x.index.tolist()[0]).reset_index()
first_data2.rename(columns={0:'idx_org'}, inplace=True)
first_data = pd.concat([first_data1, first_data2['idx_org']], axis=1)
df2md(first_data.head())

month|day|hour|minute|dir|pow|mean1|mean10|year|dateAndTime|indexMinutes|idx_org
---|---|---|---|---|---|---|---|---|---|---|---
1|18|0|0|1|0|10.355213963333378|9.092461652000043|2019|2019-01-18 00:00:00|0|0
1|19|0|0|0|0|2.5600429900000847|2.4455508209999963|2019|2019-01-19 00:00:00|1440|1440
1|20|0|0|0|0|12.467226760000047|12.896619636000034|2019|2019-01-20 00:00:00|2880|2880
1|21|0|0|1|0|14.329354940000181|14.847449557333368|2019|2019-01-21 00:00:00|4320|4320
1|22|0|0|0|0|11.481346296666917|11.841086428333323|2019|2019-01-22 00:00:00|5760|5750


#### 9. Capture Matrix Construction
* **Requirements**

In [10]:
# global variables
vCutIn = 4                    # Cut-in velocity
vRated = 12                      # Rated velocity
block_size = 10                  # Data unit block size

vMax = int(vRated + 4)
vMin = int(np.floor(vCutIn))
bin_num = vMax - vMin  # how many bins are in capture matrix
CM_opt_idx = np.inf
CM_opt = np.inf
CM_opt_array = []

In [11]:
# Capture Matrix Table

CM_table = np.zeros((5, bin_num))
CM_table[0] = np.arange(vMin, vMax)
CM_table[0][0] = vCutIn
CM_table[1] = np.arange(vMin+1, vMax+1)
CM_table[2] = np.array([20]*(bin_num-2) + [10]*2)          # Capture Matrix Target

print('Capture Matrix : \n', CM_table[0:3])
bin_array = CM_table[0:2]

Capture Matrix : 
 [[ 4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15.]
 [ 5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16.]
 [20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 10. 10.]]


* **Data Screening: wind speed, power, direction**

In [12]:
df_valid = data.loc[data['mean10'] < 16].loc[data['mean10'] >= 4]
df_valid = df_valid.loc[df_valid['pow'] * df_valid['dir'] == 1]
print(df_valid.index.size)
df2md(df_valid.head())

import copy
data = copy.deepcopy(df_valid)

14564


month|day|hour|minute|dir|pow|mean1|mean10|year|dateAndTime|indexMinutes
---|---|---|---|---|---|---|---|---|---|---
1|18|16|53|1|1|5.098772369999969|5.1713226389999845|2019|2019-01-18 16:53:00|1013
1|18|16|54|1|1|4.862361396666568|5.1015869326666605|2019|2019-01-18 16:54:00|1014
1|18|16|55|1|1|5.205255633333375|5.140628235333339|2019|2019-01-18 16:55:00|1015
1|18|16|56|1|1|5.734952640000007|5.118339111333343|2019|2019-01-18 16:56:00|1016
1|18|16|57|1|1|5.699770606666643|5.051514756666676|2019|2019-01-18 16:57:00|1017


* **Functions for Capture Matrix construction**

In [13]:
def get_iM(date_order):
    return first_data['indexMinutes'].loc[date_order].tolist()

def findnext(iM):
    data_next = data.loc[data['indexMinutes'] == iM]
    while data_next.index.size == 0:
        iM += 10
        data_next = data.loc[data['indexMinutes'] == iM]
        
        if iM > data['indexMinutes'].tail(1).tolist()[0]:
            print('### Exceed End of File {} ###'.format(iM))
            return data_next
        
    return data_next

In [14]:
result = findnext(get_iM(0))
df2md(result)

month|day|hour|minute|dir|pow|mean1|mean10|year|dateAndTime|indexMinutes
---|---|---|---|---|---|---|---|---|---|---
1|18|17|0|1|1|4.752068026666679|4.830901898999994|2019|2019-01-18 17:00:00|1020


* **Capture Matrix construction by IEC61400-13**

In [15]:
def IEC61400(date_order):
    data_next = findnext(get_iM(date_order))
    iM = data_next['indexMinutes'].tolist()[0]
    print('### indexMinutes: initial={}, actual={}'.format(first_data['indexMinutes'].loc[date_order], iM))
    
    cm = 0
    while cm == 0:
        
        if date_order > first_data.index.size:
            print('### Exceeds data size limitation {} ###'.format(date_order))
            return data_next
        
        elif False not in (CM_table[3] >= CM_table[2]):
            print('> Capture Matrix = {}'.format(CM_table[3]))
            print('> No. of elements = {:d}'.format(int(np.array(CM_table[3]).sum())))
            print('> @{}, iM={}\n'.format(data_next['dateAndTime'].tolist()[0], data_next['indexMinutes'].tolist()[0]))
            exit(0)
            return data_next
        
        if data_next.index.size > 0:
            vWind = data_next['mean10'].tolist()
            iM = data_next['indexMinutes'].tolist()[0]
            #print('vWind = ', vWind)

            no_bin = np.nonzero((bin_array[0] <= vWind) == (bin_array[1] > vWind))[0][0]
            CM_table[3][no_bin] += 1
            #print('Capture Matrix under construction:\n', CM_table[3])
            #print('indexMinutes= ', iM)
            iM += 10
            data_next = findnext(iM)
            

In [16]:
with open('IEC61400.csv', 'w') as f:
    f.write('#date_order,\
            Init_dateAndTime,\
            Init_indexMinutes,\
            Fin_dateAndTime,\
            Fin_indexMinutes,\
            duration\n')
    for i in range(14):
        CM_table[3] = np.zeros(bin_num)
        print('### date #{}: {}'.format(i, first_data['dateAndTime'].loc[i]))
        result = IEC61400(i)
        if result.index.size != 0:
            f.write('{},{},{},{},{},{}\n'.format(i,
                                              first_data['dateAndTime'].loc[i], 
                                              first_data['indexMinutes'].loc[i],
                                              result['dateAndTime'].tolist()[0], 
                                              result['indexMinutes'].tolist()[0],
                                              result['indexMinutes'].tolist()[0] - first_data['indexMinutes'].loc[i] ))

### date #0: 2019-01-18 00:00:00
### indexMinutes: initial=0, actual=1020
> Capture Matrix = [20. 52. 55. 91. 87. 80. 69. 21. 34. 63. 19. 19.]
> No. of elements = 610
> @2019-02-04 17:50:00, iM=25550

### date #1: 2019-01-19 00:00:00
### indexMinutes: initial=1440, actual=2100
> Capture Matrix = [20. 53. 55. 91. 87. 80. 69. 21. 34. 63. 19. 19.]
> No. of elements = 611
> @2019-02-04 18:10:00, iM=25570

### date #2: 2019-01-20 00:00:00
### indexMinutes: initial=2880, actual=4830
> Capture Matrix = [20. 66. 58. 94. 87. 80. 69. 21. 34. 63. 19. 19.]
> No. of elements = 630
> @2019-02-07 06:40:00, iM=29200

### date #3: 2019-01-21 00:00:00
### indexMinutes: initial=4320, actual=4830
> Capture Matrix = [20. 66. 58. 94. 87. 80. 69. 21. 34. 63. 19. 19.]
> No. of elements = 630
> @2019-02-07 06:40:00, iM=29200

### date #4: 2019-01-22 00:00:00
### indexMinutes: initial=5760, actual=6290
> Capture Matrix = [20. 67. 61. 98. 91. 83. 71. 20. 35. 63. 19. 19.]
> No. of elements = 647
> @2019-02-07 10:

#### 10. Result File Validation

In [17]:
result = pd.read_csv('IEC61400.csv')
df2md(result)

#date_order|            Init_dateAndTime|            Init_indexMinutes|            Fin_dateAndTime|            Fin_indexMinutes|            duration
---|---|---|---|---|---
0|2019-01-18 00:00:00|0|2019-02-04 17:50:00|25550|25550
1|2019-01-19 00:00:00|1440|2019-02-04 18:10:00|25570|24130
2|2019-01-20 00:00:00|2880|2019-02-07 06:40:00|29200|26320
3|2019-01-21 00:00:00|4320|2019-02-07 06:40:00|29200|24880
4|2019-01-22 00:00:00|5760|2019-02-07 10:50:00|29450|23690
5|2019-01-23 00:00:00|7200|2019-02-07 10:50:00|29450|22250
6|2019-01-24 00:00:00|8640|2019-02-07 11:50:00|29510|20870
7|2019-01-25 00:00:00|10080|2019-02-07 11:50:00|29510|19430
8|2019-01-26 00:00:00|11520|2019-02-07 14:30:00|29670|18150
9|2019-01-27 00:00:00|12960|2019-02-07 21:00:00|30060|17100
10|2019-01-28 00:00:00|14400|2019-02-07 21:00:00|30060|15660
11|2019-01-29 00:00:00|15840|2019-02-07 21:40:00|30100|14260
12|2019-01-30 00:00:00|17280|2019-02-08 17:10:00|31270|13990
13|2019-01-31 00:00:00|18720|2019-02-08 23:30:00|31650|12930
